In [ ]:
import numpy as np
import pandas as pd

In [ ]:
old_data = pd.read_csv('../Data/imputed_data_new.csv')

In [ ]:
new_data = pd.read_csv('../Data/data_new_raw_full_4.csv')

In [ ]:
data_before_2018 = new_data[new_data['year'] < 2018]
data_after_2017 = new_data[new_data['year'] >= 2018]

In [ ]:
data_before_2018['tic'].nunique(), data_after_2017['tic'].nunique()

In [ ]:
tics_only_after_2017 = set(data_after_2017['tic'].unique()) - set(data_before_2018['tic'].unique())
len(tics_only_after_2017)

In [ ]:
data_after_2017[data_after_2017['tic'].isin(tics_only_after_2017)][['tic', 'year', 'quarter', 'pastrating']]

In [ ]:
len(old_data), len(new_data)

In [ ]:
old_data.head()

In [ ]:
new_data.head()

In [ ]:
# Create the columns 'fic_CHE', 'fic_IRL', 'fic_USA' in new_data

new_data['fic_CHE'] = np.nan
new_data['fic_IRL'] = np.nan
new_data['fic_USA'] = np.nan

In [ ]:
# Drop all columns in new_data that are not in old_data

new_data = new_data[old_data.columns]

In [ ]:
new_data.head()

In [ ]:
# How many nans are in new_data?

new_data.isna().sum().sum()

In [ ]:
# Are any duplicates in new_data?

new_data.duplicated().sum()

In [ ]:
# drop duplicates in new_data

new_data = new_data.drop_duplicates()

In [ ]:
new_data_cpy = new_data.copy()
# Drop all rows where year is 2010, 2011, 2012

In [ ]:
# For every row in old_data, find the corresponding row where tic=tic, year=year, quarter=quarter in new_data and overwrite the values in new_data with the values in old_data

for i in range(len(old_data)):
    tic = old_data.loc[i, 'tic']
    year = old_data.loc[i, 'year']
    quarter = old_data.loc[i, 'quarter']
    row = new_data[(new_data['tic'] == tic) & (new_data['year'] == year) & (new_data['quarter'] == quarter)]
    if len(row) == 1:
        new_data.loc[row.index[0]] = old_data.loc[i]
    else:
        print("More than one row found")
        print(row)

In [ ]:
new_data.isna().sum().sum()

In [ ]:
len(new_data)

In [ ]:
# Get binary columns and object columns
binary_columns = new_data.columns[new_data.nunique() == 2]
object_columns = new_data.select_dtypes(include='object').columns

In [ ]:
print(binary_columns)

In [ ]:
print(object_columns)

In [ ]:
new_data["spcsrc"].value_counts()

In [ ]:
# For all rows where spcsrc is not a number map it to {"A+": 1.0, "A": 2.0, "A-": 3.0, "B+": 4.0, "B": 5.0, "B-": 6.0, "C": 7.0}

new_data["spcsrc"] = new_data["spcsrc"].replace({"A+": 1.0, "A": 2.0, "A-": 3.0, "B+": 4.0, "B": 5.0, "B-": 6.0, "C": 7.0, "D": 8.0})

In [ ]:
new_data["spcsrc"].value_counts()

In [ ]:
# Get all rows where spcsrc is not in range (1.0, 7.0)

new_data[new_data["spcsrc"] < 1.0]

In [ ]:
# For all rows where sprsrc is not in range (1.0, 7.0), set it to nan

new_data.loc[new_data["spcsrc"] < 1.0, "spcsrc"] = np.nan

In [ ]:
new_data[(new_data["tic"] == "SUN")][["tic", "pastrating", "spcsrc", "year", "quarter"]].sort_values(by=["year", "quarter"])

In [ ]:
# For all rows where tic is SUN and spcrc is nan, set spcsrc to 6.0

new_data.loc[(new_data["tic"] == "SUN") & (new_data["spcsrc"].isna()), "spcsrc"] = 6.0

In [ ]:
new_data[(new_data["tic"] == "SUN")][["tic", "pastrating", "spcsrc", "year", "quarter"]].sort_values(by=["year", "quarter"])

In [ ]:
# What columns contain nans?

new_data.columns[new_data.isna().any()]

In [ ]:
new_data["costat"] = new_data["costat"].replace({"I": 0.0, "A": 1.0})
new_data["curncdq"] = new_data["curncdq"].replace({"USD": 1.0, "EUR": 0.0})

In [ ]:
binary_columns = new_data.columns[new_data.nunique() == 2]
object_columns = new_data.select_dtypes(include='object').columns

In [ ]:
print(binary_columns)

In [ ]:
print(object_columns)

In [ ]:
# Check if datadate and tic have any nans

new_data[['datadate', 'tic', 'pastrating']].isna().sum()

In [ ]:
# Are there any columns that are only nans?

new_data.columns[new_data.isna().all()]

In [ ]:
new_data.value_counts()

In [ ]:
new_data.describe()

In [ ]:
new_data.isna().sum()

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import RobustScaler

def iterative_impute(X_train, X_test, max_iter=10):
    imputer = IterativeImputer(estimator=BayesianRidge(), max_iter=max_iter, random_state=42, verbose=2, min_value=X_train.min(), max_value=X_train.max())
    imputer.fit(X_train)
    imputed_data = imputer.transform(X_test)
    imputed_df = pd.DataFrame(imputed_data, columns=X_test.columns)
    return imputed_df

In [ ]:
new_data = new_data.sort_values(by='datadate').reset_index(drop=True)

In [ ]:
# Print dtypes

new_data.dtypes

In [ ]:
# Change the dtype of curncdq, costat, spcsrc to float

new_data['curncdq'] = new_data['curncdq'].astype(float)
new_data['costat'] = new_data['costat'].astype(float)
new_data['spcsrc'] = new_data['spcsrc'].astype(float)

In [ ]:
# Create a new df with no nans

new_data_no_nans = new_data.dropna()
new_data_only_nans = new_data[new_data.isna().any(axis=1)]

In [ ]:
new_data_no_nans.isna().sum().sum()

In [ ]:
new_data_only_nans.isna().sum().sum()

In [ ]:
new_data_only_nans

In [ ]:
new_data_only_nans.isna().sum()

In [ ]:
# Get all columns except datadate and tic

cols_to_impute = new_data_no_nans.columns.difference(['datadate', 'tic', 'year', 'quarter', 'pastrating'])
X_train = new_data_no_nans[cols_to_impute].copy()
X_test = new_data_only_nans[cols_to_impute].copy()


In [ ]:
X_train.shape

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
X_train.isna().sum().sum(), X_test.isna().sum().sum()

In [ ]:

imputed_df = iterative_impute(X_train, X_test, 100)

In [ ]:
new_data.describe()

In [ ]:
imputed_df.describe()

In [ ]:
imputed_df.isna().sum().sum()

In [ ]:
new_data.shape, imputed_df.shape

In [ ]:
new_data_no_nans.index

In [ ]:
new_data_only_nans.index

In [ ]:
imputed_df.index = new_data_only_nans.index

new_data_only_nans.update(imputed_df)

In [ ]:
new_data_only_nans.isna().sum().sum()

In [ ]:
new_data.isna().sum().sum()

In [ ]:
new_data_full = pd.concat([new_data_no_nans, new_data_only_nans])
new_data_full = new_data_full.sort_values(by='datadate').reset_index(drop=True)

In [ ]:
new_data_full.isna().sum().sum()

In [ ]:
new_data_full.shape, new_data.shape

In [ ]:
# For every tic, check if they have entries for all quarters in every year from 2010 to 2020

for tic in new_data_full['tic'].unique():
    tic_data = new_data_full[new_data_full['tic'] == tic]
    for year in range(2010, 2021):
        for quarter in range(1, 5):
            if len(tic_data[(tic_data['year'] == year) & (tic_data['quarter'] == quarter)]) == 0:
                print(f"No entry for tic: {tic}, year: {year}, quarter: {quarter}")

In [ ]:
# For every quarter in 2018 to 2020, print how many tics have entries

for year in range(2018, 2021):
    for quarter in range(1, 5):
        print(f"Year: {year}, Quarter: {quarter}, Tics: {len(new_data_full[(new_data_full['year'] == year) & (new_data_full['quarter'] == quarter)]['tic'].unique())}")

In [ ]:
# # For each value in 'curncdq', 'costat', 'fic_CHE', 'fic_IRL', 'fic_USA' that is not 0 or 1, set it to 0 or 1 depending on which value is closer

new_data_full['curncdq'] = new_data_full['curncdq'].apply(lambda x: 0 if x < 0.5 else 1)
new_data_full['costat'] = new_data_full['costat'].apply(lambda x: 0 if x < 0.5 else 1)
new_data_full['fic_CHE'] = new_data_full['fic_CHE'].apply(lambda x: 0 if x < 0.5 else 1)
new_data_full['fic_IRL'] = new_data_full['fic_IRL'].apply(lambda x: 0 if x < 0.5 else 1)
new_data_full['fic_USA'] = new_data_full['fic_USA'].apply(lambda x: 0 if x < 0.5 else 1)

In [ ]:
# What dtypes are datadate and tic?

new_data_full[['datadate', 'tic']].dtypes

In [ ]:
# Print all binary columns

binary_columns = new_data_full.columns[new_data_full.nunique() == 2]
binary_columns

In [ ]:
new_data_full.describe()

In [ ]:
new_data_full

In [ ]:
new_data_full.isna().sum().sum()

In [ ]:
# Check for inf values

new_data_full[new_data_full == np.inf].sum().sum()

In [ ]:
new_data_full.to_csv("../Data/imputed_data_new_3.csv", index=False)